In [25]:
import snowflake.connector
import requests
import openai
import base64
from openai import OpenAI

In [6]:
# Snowflake credentials
SNOWFLAKE_ACCOUNT=''
SNOWFLAKE_USER=''
SNOWFLAKE_PASSWORD=''
SNOWFLAKE_DATABASE=''
SNOWFLAKE_SCHEMA=''
SNOWFLAKE_WAREHOUSE=''
SNOWFLAKE_ROLE=''
SNOWFLAKE_TABLE = ""

In [8]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

# Fetch records from the table
cursor = conn.cursor()
cursor.execute(f"SELECT TITLE, DESCRIPTION, FULL_ADDRESS, EVENT_DATES, IMAGE_URL FROM {SNOWFLAKE_TABLE}")
records = cursor.fetchall()

In [31]:
def get_image_description(image_url):
    """Download image and get a textual description using OpenAI Vision model"""
    response = requests.get(image_url)
    if response.status_code == 200:
        base64_image = base64.b64encode(response.content).decode('utf-8')
        
        # OpenAI API Call
        client = OpenAI(api_key='')
        response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "You will be provided with an image related to an event. Describe the image concisely, focusing only on event-specific details. If the image includes information such as the event name, timing, date, or price, extract and provide those details. Exclude any unrelated observations or interpretations.",
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )


        return response.choices[0].message.content
    else:
        return "Image not available or failed to load."

### Classify events

In [35]:
def classify_event(title, description, address, dates, image_description):
    """Send event details to OpenAI for classification into predefined categories"""

    categories = [
        "Concert", "Sports", "Festival", "Exhibition", "Theater", "Comedy Show", 
        "Food & Drink", "Networking", "Educational", "Family-Friendly", "Tech Conference", "Other"
    ]

    prompt_message = f"""
    Based on the given event details, classify the event into one or more categories from the following list:
    {', '.join(categories)}.

    Event Details:
    - Title: {title}
    - Description: {description}
    - Address: {address}
    - Event Dates: {dates}
    - Image Description: {image_description}

    Return only the category names as a list.
    """
    client = OpenAI(api_key='')
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an AI that classifies events based on their details."},
            {"role": "user", "content": prompt_message}
        ],
        max_tokens=100
    )

    return response.choices[0].message.content.strip()


In [37]:
# Prepare data for update
updated_records = []

for record in records:
    title, description, address, dates, image_url = record

    # Get image description
    image_description = get_image_description(image_url)

    # Get event category
    event_category = classify_event(title, description, address, dates, image_description)

    # Append to list
    updated_records.append((title, description, address, dates, image_url, image_description, event_category))


In [41]:
# Insert new data back into Snowflake 
cursor.executemany(
    f"""
    UPDATE {SNOWFLAKE_TABLE} 
    SET IMAGE_DESCRIPTION = %s, EVENT_CATEGORY = %s
    WHERE TITLE = %s AND DESCRIPTION = %s AND FULL_ADDRESS = %s AND EVENT_DATES = %s AND IMAGE_URL = %s
    """,
    [(img_desc, category, title, desc, addr, dates, img_url) for title, desc, addr, dates, img_url, img_desc, category in updated_records]
)

# Commit and close connection
conn.commit()
cursor.close()
conn.close()